<a href="https://colab.research.google.com/github/Billyjr2003/deep-learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torch.nn.functional as fc
from torch.utils.data import dataloader

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [67]:
train_data = dataset.MNIST(root='dataset/', train= True, transform= transforms.ToTensor(), download= True)
test_data = dataset.MNIST(root='dataset/', train= False, transform= transforms.ToTensor(), download= True)

In [68]:
# Hyperparameters
input_size = 28
num_seq = 28
hidden_size = 256
num_layers = 2
num_classes = 10
batch_size = 64
l_r = 0.001
num_epochs = 5

In [69]:
train_data_loader = dataloader.DataLoader(dataset= train_data, batch_size= batch_size, shuffle= True)
test_data_loader = dataloader.DataLoader(dataset= test_data, batch_size= batch_size, shuffle= False)

In [70]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ = self.rnn(x, h_0)
    out = self.fc(out[:, -1, :])
    return out

In [71]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [72]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= l_r)

In [73]:
def accuarcy(loader, model_cls, device):
  num_correct = 0
  num_samples = 0
  model_cls.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)
      scores = model_cls(x)
      predections = torch.argmax(scores, dim = 1)
      num_correct += (predections == y).sum()
      num_samples += predections.size(0)
    print(f'Got {num_correct / num_samples} with accuracy {(float(num_correct)/ float(num_samples))* 100} %')
  model_cls.train()

**RNN Model**

In [74]:
for epoch in range(num_epochs):
  for batch , (data, target) in enumerate(train_data_loader):
    data = data.to(device).squeeze(1)
    target = target.to(device)
    pred = model(data)

    loss_fn = loss(pred, target)

    optimizer.zero_grad()
    loss_fn.backward()
    optimizer.step()

accuarcy(loader=train_data_loader, model_cls= model, device= device)
accuarcy(loader=test_data_loader, model_cls= model, device= device)

Got 0.9551500082015991 with accuracy 95.515 %
Got 0.9529999494552612 with accuracy 95.3 %
